This notebook contains code for performing linear fits (to logarithmic data, so equivalent
to power-law fits to the original linear data) using code from the Scipy package in Python.

Specifically, this performs fits to the size of inner bars and nuclear rings as functions of 
galaxy stellar mass or (outer or single) bar size.

# Requirements

This notebook is meant to be run within the full **db-nr_paper** repository, including the associated data files.

# Setup

## Initial Imports, Path and Global Variable Definitions

In [1]:
import sys, datetime
import numpy as np
from scipy.optimize import curve_fit
import scipy.stats

# CHANGE THIS TO POINT TO APPROPRIATE LOCAL DIRECTORY (DEFAULT = SAME DIRECTORY AS THIS NOTEBOOK)
# baseDir = "/Users/erwin/Documents/Working/Papers/Paper-extended-wiyn-survey/public/"
baseDir = os.getcwd() + "/"
dataDir = baseDir + "data/"
sys.path.append(baseDir)

import datautils as du
import fitting_barsizes
import dbnr_utils

fitDict = {}
fitParamNamesDict = {}

## Defining different subsamples via index vectors

Lists of integers defining indices of galaxies in Parent Disc Sample which meet various criteria
that define specific subsamples.

## Define Data Vectors and Subsamples

In [2]:
# NEW
df_bars, gnames_bars_rowdict = dbnr_utils.GetBarredGalaxyData("table_mainsample.dat", dataDir)
nBarredGalaxies = len(df_bars.name)

In [3]:
# assume a constant fractional uncertainty of 10 per cent for inner-bar and nuclear-ring sizes (0.044 in logarithmic terms)
log_errs = np.zeros(nBarredGalaxies) + 0.044

# indices specifying DB or NR galaxies
ii_db = [i for i in range(nBarredGalaxies) if df_bars.dbFlag[i]]
ii_nr = [i for i in range(nBarredGalaxies) if df_bars.nrFlag[i]]

# log of outer- or single-bar size in kpc; log of inner-bara size in kpc; log of NR size in kpc
logamax_kpc = np.log10(df_bars.amax_dp_kpc)
logamax_inner_kpc = np.log10(df_bars.amax2_dp_kpc)
lognr_kpc = np.log10(df_bars.nr_dp_kpc)

/var/folders/rj/3r6_hsl93l737byvn_vy_tmm0000gp/T/ipykernel_88793/1392126389.py:10: RuntimeWarning: divide by zero encountered in log10
  logamax_inner_kpc = np.log10(df_bars.amax2_dp_kpc)
/var/folders/rj/3r6_hsl93l737byvn_vy_tmm0000gp/T/ipykernel_88793/1392126389.py:11: RuntimeWarning: divide by zero encountered in log10
  lognr_kpc = np.log10(df_bars.nr_dp_kpc)


# Spearman Correlation Coefficients

In [4]:
r,p = scipy.stats.spearmanr(df_bars.amax_dp_kpc[ii_db], df_bars.amax2_dp_kpc[ii_db])
print(r,p)

0.7588709677419356 7.508273355480878e-07


In [5]:
r,p = scipy.stats.spearmanr(df_bars.logmstar[ii_db], df_bars.amax2_dp_kpc[ii_db])
print(r,p)

0.5978425274862617 0.0003827874887619227


# Performing Fits

## Define Useful Functions

In [6]:
# wrappers for functions, in form needed by s4gutils.bootstrap_validation

def nicelin( x, p ):
    return fitting_barsizes.flin(x, *p)

def dofit_lin( x, y, p0, errs ):
    pp,pcov = curve_fit(fitting_barsizes.flin, x, y, p0=p0, sigma=errs)
    return pp

def nicebrokenlin( x, p ):
    return fitting_barsizes.fbrokenlin(x, *p)

def dofit_brokenlin( x, y, p0, errs ):
    pp,pcov = curve_fit(fitting_barsizes.fbrokenlin, x, y, p0=p0, sigma=errs)
    return pp


def nicecomposite( x, p ):
    return fitting_barsizes.fmulti_lin_brokenlin(x, *p)

def nicemultilin( x, p ):
    return fitting_barsizes.fmulti_lin(x, *p)

def dofit_composite( x, y, p0, errs ):
    pp,pcov = curve_fit(fitting_barsizes.fmulti_lin, x, y, p0=p0, sigma=errs)
    return pp


def nicecomposite_binary( x, p ):
    return fitting_barsizes.fmulti_binary(x, *p)

def dofit_composite_binary( x, y, p0, errs ):
    pp,pcov = curve_fit(fitting_barsizes.fmulti_binary, x, y, p0=p0, sigma=errs)
    return pp


def PrintFitResultsToString( fitDict, key ):
    """
    Ex.:
    PrintParameters(fitDict, "inner-barsize-vs-Mstar")
    """
    key_uncertainties = key + "_confint"
    fitResult = fitDict[key]
    params, aic, mse = fitResult
    txt2 = "\n\t AIC = {0:.2f}, Adjusted test MSE = {1:.3f}".format(aic, mse)
    
    if len(params) == 2:
        alpha = params[0]
        beta = params[1]
        ci_alpha, ci_beta = fitDict[key_uncertainties]
        alpha_max, alpha_min = np.max(ci_alpha), np.min(ci_alpha)
        beta_max, beta_min = np.max(ci_beta), np.min(ci_beta)
        alpha_up = alpha_max - alpha
        alpha_low = alpha - alpha_min
        beta_up = beta_max - beta
        beta_low = beta - beta_min
        txt1 = "\t alpha, beta = {0:.3f} +{1:.3f}/-{2:.3f}, {3:.3f} +{4:.3f}/-{5:.3f}".format(alpha,
                            alpha_up, alpha_low, beta, beta_up, beta_low)
    else:   # 3 params
        alpha = params[0]
        beta1 = params[1]
        beta2 = params[2]
        ci_alpha, ci_beta1, ci_beta2 = fitDict[key_uncertainties]
        alpha_max, alpha_min = np.max(ci_alpha), np.min(ci_alpha)
        beta1_max, beta1_min = np.max(ci_beta1), np.min(ci_beta1)
        beta2_max, beta2_min = np.max(ci_beta2), np.min(ci_beta2)
        alpha_up = alpha_max - alpha
        alpha_low = alpha - alpha_min
        beta1_up = beta1_max - beta1
        beta1_low = beta1 - beta1_min
        beta2_up = beta2_max - beta2
        beta2_low = beta2 - beta2_min
        txt1 = "\t alpha, beta1, beta2 = {0:.3f} +{1:.3f}/-{2:.3f}, {3:.3f} +{4:.3f}/-{5:.3f}, {6:.3f} +{7:.3f}/-{8:.3f}".format(alpha,
                            alpha_up, alpha_low, beta1, beta1_up, beta1_low, beta2, beta2_up, beta2_low)

    return txt1 + txt2


## Inner-bar and Nuclear-ring Size vs Stellar Mass and Outer-Bar Size

**NOTE:** Fit results (coefficient values, AIC, etc.) in this section vary slightly from those in the published version 
of the paper, because for simplicity the fits done here use NR and bar sizes in kpc computed with the truncated versions of galaxy distances from `table_mainsample.dat` (e.g., for NGC 4735, $D = 5.1$ Mpc in `table_mainsample.dat`, while $D = 5.126$ Mpc in the original calculations).

The differences are much smaller than the estimated uncertainties (e.g., inner-bar size fit vs stellar mass: $\alpha = -5.982$ +1.525/$-$1.604 here, versus $-5.98 \pm 1.50$ in Table 5 of the paper.)


### Inner Bar Sizes

#### Inner Bar Size vs Stellar Mass

In [7]:
xx, yy = df_bars.logmstar, logamax_inner_kpc

p_init = [-5.5, 0.3]
params_ib_vs_mstar, aic_ib_vs_mstar = fitting_barsizes.DoFit(xx, yy, log_errs, ii_db, p_init)
mse = dbnr_utils.bootstrap_validation(xx[ii_db], yy[ii_db], 1000, dofit_lin, computeModelFn=nicelin, initialParams=params_ib_vs_mstar, errs=log_errs[ii_db], verbose=True)
fitDict['inner-barsize-vs-Mstar'] = [params_ib_vs_mstar, aic_ib_vs_mstar, mse]
fitDict['inner-barsize-vs-Mstar_confint'] = fitting_barsizes.ParameterUncertainties(xx, yy, log_errs, ii_db, params_ib_vs_mstar, nIterations=2000) 
fitParamNamesDict['inner-barsize-vs-Mstar'] = ["alpha", "beta"]

txt = PrintFitResultsToString(fitDict, "inner-barsize-vs-Mstar")
print(txt)

   alpha, beta = [-5.98167, 0.518671]
   AIC = 628.372
[-5.98167402  0.51867119]
training MSE = 0.0389663
test MSE = 0.0478762 (1000 successful iterations)
Adjusted test MSE = 0.0445974
	 alpha, beta = -5.982 +1.470/-1.631, 0.519 +0.155/-0.139
	 AIC = 628.37, Adjusted test MSE = 0.045


#### Inner Bar Size vs Outer Bar Size

In [8]:
xx,yy = logamax_kpc, logamax_inner_kpc

p_init = [-5.5, 0.3]
params_ib_vs_ob, aic_ib_vs_ob = fitting_barsizes.DoFit(xx, yy, log_errs, ii_db, p_init)
mse = dbnr_utils.bootstrap_validation(xx[ii_db], yy[ii_db], 1000, dofit_lin, computeModelFn=nicelin, initialParams=params_ib_vs_ob, errs=log_errs[ii_db], verbose=True)
fitDict['inner-barsize-vs-logamax'] = [params_ib_vs_ob, aic_ib_vs_ob, mse]
fitDict['inner-barsize-vs-logamax_confint'] = fitting_barsizes.ParameterUncertainties(xx, yy, log_errs, ii_db, params_ib_vs_ob, nIterations=2000) 
fitParamNamesDict['inner-barsize-vs-logamax'] = ["alpha", "beta"]

txt = PrintFitResultsToString(fitDict, "inner-barsize-vs-logamax")
print(txt)

   alpha, beta = [-0.90022, 0.737922]
   AIC = 389.009
[-0.90022021  0.73792161]
training MSE = 0.0240177
test MSE = 0.0279366 (1000 successful iterations)
Adjusted test MSE = 0.0264944
	 alpha, beta = -0.900 +0.051/-0.054, 0.738 +0.089/-0.089
	 AIC = 389.01, Adjusted test MSE = 0.026


#### Nuclear Ring Size vs Stellar Mass

In [9]:
# Nuclear ring size as a function of galaxy stellar mass
xx,yy = df_bars.logmstar, lognr_kpc

p0 = [-5.5, 0.3]
params_nr_vs_mstar, aic_nr_vs_mstar = fitting_barsizes.DoFit(xx, yy, log_errs, ii_nr, p0)
mse = dbnr_utils.bootstrap_validation(xx[ii_nr], yy[ii_nr], 1000, dofit_lin, computeModelFn=nicelin, initialParams=params_nr_vs_mstar, errs=log_errs[ii_nr], verbose=True)
fitDict['nrsize-vs-Mstar'] = [params_nr_vs_mstar, aic_nr_vs_mstar, mse]
fitDict['nrsize-vs-Mstar_confint'] = fitting_barsizes.ParameterUncertainties(xx, yy, log_errs, ii_nr, params_nr_vs_mstar, nIterations=2000) 
fitParamNamesDict['nrsize-vs-Mstar'] = ["alpha", "beta"]

txt = PrintFitResultsToString(fitDict, "nrsize-vs-Mstar")
print(txt)

   alpha, beta = [-6.78296, 0.60582]
   AIC = 969.167
[-6.78295866  0.60581966]
training MSE = 0.0602495
test MSE = 0.0758954 (1000 successful iterations)
Adjusted test MSE = 0.0701377
	 alpha, beta = -6.783 +2.002/-2.071, 0.606 +0.198/-0.192
	 AIC = 969.17, Adjusted test MSE = 0.070


#### Nuclear Ring Size vs (Outer or Single) Bar Size

In [10]:
# Nuclear ring size as a function of (outer or single) bar size
xx,yy = np.log10(df_bars.amax_dp_kpc), lognr_kpc

p0 = [-5.5, 0.3]
params_nr_vs_ob, aic_nr_vs_ob = fitting_barsizes.DoFit(xx, yy, log_errs, ii_nr, p0)
mse = dbnr_utils.bootstrap_validation(xx[ii_nr], yy[ii_nr], 1000, dofit_lin, computeModelFn=nicelin, initialParams=params_nr_vs_ob, errs=log_errs[ii_nr], verbose=True)
fitDict['nrsize-vs-logamax'] = [params_nr_vs_ob, aic_nr_vs_ob, mse]
fitDict['nrsize-vs-logamax_confint'] = fitting_barsizes.ParameterUncertainties(xx, yy, log_errs, ii_nr, params_nr_vs_ob, nIterations=2000) 
fitParamNamesDict['nrsize-vs-logamax'] = ["alpha", "beta"]

txt = PrintFitResultsToString(fitDict, "nrsize-vs-logamax")
print(txt)

   alpha, beta = [-0.88005, 0.820183]
   AIC = 994.427
[-0.88004957  0.82018315]
training MSE = 0.061827
test MSE = 0.0729617 (1000 successful iterations)
Adjusted test MSE = 0.0688641
	 alpha, beta = -0.880 +0.117/-0.163, 0.820 +0.238/-0.163
	 AIC = 994.43, Adjusted test MSE = 0.069


## Save Best-Fit Parameter values, AIC, MSE

In [11]:
dictKeys = list(fitDict.keys())
param_names = [k for k in dictKeys if k.find("_confint") < 0]
param_limit_names = [k for k in dictKeys if k.find("_confint") > 0]

### Save best-fit parameter values to file

In [12]:
def WriteParams( outf, name, p_vector ):
    txt = "%s: " % name
    nPts = len(p_vector)
    for i in range(nPts):
        txt += "\t%.4f" % p_vector[i]
    outf.write(txt + "\n")

In [13]:
dictKeys

['inner-barsize-vs-Mstar',
 'inner-barsize-vs-Mstar_confint',
 'inner-barsize-vs-logamax',
 'inner-barsize-vs-logamax_confint',
 'nrsize-vs-Mstar',
 'nrsize-vs-Mstar_confint',
 'nrsize-vs-logamax',
 'nrsize-vs-logamax_confint']

In [14]:
ts = '{:%Y-%b-%d %H:%M:%S}'.format(datetime.datetime.now())
bestfitParamsFile = baseDir + "bestfit_parameters.txt"
outf = open(bestfitParamsFile, 'w')
outf.write("# Best-fit parameters for S4G barsize analysis (barsize_fits.ipynb): %s\n" % ts)
outf.write("# Name of fit, followed by best-fit parameter values\n#\n")
for name in param_names:
    params, aic, mse = fitDict[name]
    parameterNames = fitParamNamesDict[name]
    outf.write("# %s:" % name)
    for paramName in parameterNames:
        outf.write("  %s" % paramName)
    outf.write("\n")
    WriteParams(outf, name, params)
outf.close()

### Save AIC and MSE to file

In [15]:
outf = open(baseDir + "bestfit_aic_etc.txt", 'w')
outf.write("# AIC and MSE_pred for best-fit parameters for S4G barsize analysis (barsize_fits.ipynb): %s\n" % ts)
outf.write("# [see %s for parameter values]\n" % bestfitParamsFile)
outf.write("# Name of fit, best-fit AIC, MSE_pred\n")
for name in param_names:
    params, aic, mse = fitDict[name]
    txt = "%s:\t\t%.1f\t%.4f\n" % (name, aic, mse)
    outf.write(txt)
outf.close()

### Save parameter uncertainties to file

In [16]:
outf = open(baseDir + "bestfit_parameter_uncertainties.txt", 'w')
outf.write("# Uncertainties for best-fit parameters for S4G barsize analysis (barsize_fits.ipynb): %s\n" % ts)
outf.write("# [see %s for parameter values]\n" % bestfitParamsFile)
outf.write("# Name of fit, parameter uncertainties\n")
for name in param_limit_names:
    paramLimits = fitDict[name]
    nParams = len(paramLimits)
    txt = "%s:   " % name
    for i in range(nParams - 1):
        txt += "(%.4f,%.4f), " % paramLimits[i]
    txt += "(%.4f,%.4f)\n" % paramLimits[-1]
    outf.write(txt)
outf.close()

# Extra Stuff

#### Inner Bar Size vs Stellar Mass *and* Outer Bar Size

In [17]:
xx,yy = [df_bars.logmstar, logamax_kpc], logamax_inner_kpc

p_init = [-5, 0.5, 0.5]
params_ib_vs_obmstar, aic_ib_vs_obmstar = fitting_barsizes.DoFit(xx,yy, log_errs, ii_db, p_init, "multi-linear")
xx_composite = [df_bars.logmstar[ii_db], logamax_kpc[ii_db]]
mse = astrostat.bootstrap_validation(xx_composite, yy[ii_db], 1000, dofit_composite, computeModelFn=nicemultilin, initialParams=params_ib_vs_obmstar, errs=log_errs[ii_db], verbose=True)
fitDict['inner-barsize-vs-Mstar+outer-barsize'] = [params_ib_vs_obmstar, aic_ib_vs_obmstar ,mse]
fitDict['inner-barsize-vs-Mstar+outer-barsize_confint'] = fitting_barsizes.ParameterUncertainties(xx, yy, log_errs, ii_db, params_ib_vs_obmstar, nIterations=2000, mode="multi-linear") 

   AIC = 354.076
[-2.76045307  0.18161553  0.62943899]
training MSE = 0.0224873
test MSE = 0.0295893 (1000 successful iterations)
Adjusted test MSE = 0.0269758


#### Nuclear Ring Size vs Stellar Mass *and* Outer Bar Size

In [18]:
xx,yy = [df_bars.logmstar, logamax_kpc], lognr_kpc

p_init = [-5, 0.5, 0.5]
params_nr_vs_obmstar, aic_nr_vs_obmstar = fitting_barsizes.DoFit(xx,yy, log_errs, ii_nr, p_init, "multi-linear")
xx_composite = [df_bars.logmstar[ii_nr], logamax_kpc[ii_nr]]
mse = astrostat.bootstrap_validation(xx_composite, yy[ii_nr], 1000, dofit_composite, computeModelFn=nicemultilin, initialParams=params_nr_vs_obmstar, errs=log_errs[ii_nr], verbose=True)
fitDict['nrsize-vs-Mstar+outer-barsize'] = [params_nr_vs_obmstar, aic_nr_vs_obmstar ,mse]
fitDict['nrsize-vs-Mstar+outer-barsize_confint'] = fitting_barsizes.ParameterUncertainties(xx, yy, log_errs, ii_nr, params_nr_vs_obmstar, nIterations=2000, mode="multi-linear") 

   AIC = 811.266
[-5.29992617  0.43372053  0.55994349]
training MSE = 0.0510396
test MSE = 0.0697969 (1000 successful iterations)
Adjusted test MSE = 0.0628942
